In [ ]:
import pandas as pd
import time
import tweepy
import pickle
import glob
import tweepy
import datetime
import numpy as np
import csv
import re
from time import strftime
pd.set_option('display.max_colwidth', None)

In [ ]:
result.to_csv('Fake_News_Engagement.csv')

In [ ]:
result = pd.read_csv('Fake_News_Engagement.csv')

In [ ]:
result

In [ ]:
df_manual = pd.read_csv('Fake_tweet_manual.csv')
print(len(df_manual))

In [ ]:
df_auto = pd.read_csv('Final_Fake_Accounts.csv')

In [ ]:
df_auto

In [ ]:
df_manual.columns

In [ ]:
df_auto.columns

In [ ]:
df_manual = df_manual.rename(columns = {'FactCheckNews':'Fact_Check_Link', 'NewsType':'Tweet_Topic'})

In [ ]:
df_fake_news_auto = df_auto.drop_duplicates('Fact_Check_Link')[['Fact_Check_Link', 'Tweet_Topic']].reset_index(drop = True)

In [ ]:
for index1, row1 in df_manual.iterrows():
    for index2, row2 in df_fake_news_auto.iterrows():
        if df_manual.at[index1, 'Fact_Check_Link'] == df_fake_news_auto.at[index2, 'Fact_Check_Link']:
            df_manual.at[index1, 'Tweet_Topic'] = df_fake_news_auto.at[index2, 'Tweet_Topic']
 

In [ ]:
for index1, row1 in df_manual.iterrows():
    df_manual.at[index1, 'Tweet_ID'] = str.split(df_manual.at[index1, 'ScreenName'], '/')[-1]

In [ ]:
str.split(df_manual.ScreenName[0], '/')[-1]

In [ ]:
df_auto

In [ ]:
df_manual

In [ ]:
for index1, row1 in df_manual.iterrows():
    if len(str.split(df_manual.at[index1, 'ScreenName'], '/')) > 1:
        df_manual.at[index1, 'SC'] = str.split(df_manual.at[index1, 'ScreenName'], '/')[3]
    else:
        df_manual.at[index1, 'SC'] = df_manual.at[index1, 'ScreenName']

In [ ]:
for index1, row1 in df_manual.iterrows():
     if df_manual.at[index1, 'SC'][0] == '@':
            df_manual.at[index1, 'SC'] = df_manual.at[index1, 'SC'][1:]

In [ ]:
df_manual_filter = df_manual

In [ ]:
for index1, row1 in df_auto.iterrows():
     if df_auto.at[index1, 'ScreenName'][0] == '@':
            df_auto.at[index1, 'SC'] = df_auto.at[index1, 'ScreenName'][1:]

In [ ]:
df_auto_filter = df_auto[['ScreenName', 'Fact_Check_Link', 'Tweet_Topic', 'Tweet_ID', 'SC']]

In [ ]:
df_manual_filter.drop(df_manual_filter[df_manual_filter.Fact_Check_Link == 
                     'https://www.bhaskar.com/no-fake-news/news/child-beating-video-from-egypt-circulating-as-an-incident-from-dps-rajbagh-01604931.html'].index, inplace=True)

In [ ]:
df_manual_filter

In [ ]:
frames = [df_auto_filter, df_manual_filter]

df_filter_all = pd.concat(frames)

In [ ]:
df_filter_all = df_filter_all.reset_index(drop = True)

In [ ]:
df_filter_all.groupby('Fact_Check_Link').count()

In [ ]:
df_filter_all_just_link = df_filter_all[['Fact_Check_Link','Tweet_Topic']].drop_duplicates(keep='first')

In [ ]:
df_filter_all_just_link=df_filter_all_just_link.dropna()

In [ ]:
df_filter_all_just_link.groupby('Tweet_Topic').count()

In [ ]:
df_filter_all_just_link

In [ ]:
df_filter_all

In [ ]:
df_filter_all_remove_dup =df_filter_all.drop_duplicates(subset=['Tweet_ID', 'SC'], keep = 'first').reset_index(drop = True)

In [ ]:
df_filter_all_remove_dup.loc[df_filter_all_remove_dup.Fact_Check_Link.isna()]

In [ ]:
df_total = df_filter_all_remove_dup.groupby('Fact_Check_Link').count().reset_index()

In [ ]:
sum(df_total.Tweet_ID)

In [ ]:
df_filter_all_remove_dup.iloc[3087].Tweet_Topic = df_filter_all_remove_dup.iloc[3086].Tweet_Topic
df_filter_all_remove_dup.iloc[3087].Fact_Check_Link = df_filter_all_remove_dup.iloc[3086].Fact_Check_Link

In [ ]:
df_filter_all_remove_dup.iloc[3087]

In [ ]:
df_total = df_filter_all_remove_dup.groupby('Fact_Check_Link').count().reset_index()

In [ ]:
sum(df_total.Tweet_ID)

# Fake Tweet Process

In [ ]:
file = glob.glob("U:\Twitter Research\Fake News Scraping\Fake Tweet\All\*.data")

In [ ]:
objects = []
user = []
j=0
for i in file:
    j=j+1
    with (open(i, "rb")) as openfile:
        user.append(i)
        while True:
            try:
                objects.append(pickle.load(openfile))
            except EOFError:
                break

In [ ]:
def tweet_process(alltweets):
    tweet_id = alltweets.id_str
    tweet_created = alltweets.created_at
    
    try:
        tweet_text = alltweets.full_text
    except:
        tweet_text = alltweets.text
        
    user_id = alltweets.user.id 
    screen_name_df = alltweets.user.screen_name 
    retweet_count =  alltweets.retweet_count 
    favorite_count = alltweets.favorite_count 
    lang = alltweets.lang 
    source = alltweets.source  
    
    tweet_dict = {'Tweet_ID': tweet_id, 
                  'Tweet_Time': tweet_created, 
                  'Tweet_Text': tweet_text, 
                  'Retweet_Count': retweet_count,
                  'Favorite_Count': favorite_count,
                  'Tweet_Source' : source,
                  'Tweet_Lang': lang,
                  'User_ID': user_id,
                  'User_ScreenName': screen_name_df
                  }  

    return tweet_dict
    

In [ ]:
tt= tweet_process(objects[0])
xx =pd.DataFrame(tt,index=[0])
db=xx

In [ ]:
for i in range(1, len(objects)):
    tt= tweet_process(objects[i])
    xx =pd.DataFrame(tt,index=[i])
    frames = [db, xx]
    db = pd.concat(frames)

In [ ]:
db_auto = pd.merge( db,df_filter_all_remove_dup, on = 'Tweet_ID', how ='left')
db_auto = db_auto.drop_duplicates('Tweet_ID')

In [ ]:
db_auto


In [ ]:
db_auto=db_auto.dropna()
db_auto_drop_duplicate = db_auto.drop_duplicates(subset=['Tweet_ID'], keep ='first')

In [ ]:
len(db_auto_drop_duplicate)

In [ ]:
db_auto_drop_duplicate=db_auto_drop_duplicate.reset_index()

In [ ]:
db_auto_drop_duplicate.sort_values(by='Fact_Check_Link')

In [ ]:
db_auto_drop_duplicate['Tweet_ID'].count()

In [ ]:
dbb = db_auto_drop_duplicate.groupby('Fact_Check_Link').count().reset_index()

In [ ]:
db_auto_drop_duplicate.groupby('User_ID').count().reset_index().sort_values('Tweet_ID', ascending = False)

In [ ]:
dbbx = dbb[['Fact_Check_Link', 'Tweet_ID']]

In [ ]:
dbbx = dbbx.rename(columns = {'Tweet_ID':'Available_Number_of_copies'})

In [ ]:
sum(dbbx.Available_Number_of_copies)

In [ ]:
df_filter_all_remove_dup.groupby('Fact_Check_Link').count()

In [ ]:
right =df_filter_all_remove_dup.drop_duplicates('Tweet_ID').reset_index(drop=True).groupby('Fact_Check_Link').count().sort_values(by=['Tweet_ID'], ascending=False).reset_index()[['Fact_Check_Link', 'Tweet_ID']]

In [ ]:
left=db_auto_drop_duplicate.groupby('Fact_Check_Link').sum().sort_values(by=['Retweet_Count'], ascending=False).reset_index()[['Fact_Check_Link','Retweet_Count','Favorite_Count']]

In [ ]:
result = pd.merge(left, right, on="Fact_Check_Link", how = 'outer')

In [ ]:
sum(right.Tweet_ID)

In [ ]:
result = result.rename(columns = {'Tweet_ID':'Number_of_copies'}) 

In [ ]:
result = result.sort_values(by=['Number_of_copies'], ascending=False).reset_index(drop = True)

In [ ]:
result = pd.merge(result, df_filter_all_just_link, on="Fact_Check_Link", how = 'outer')

In [ ]:
sum(result.Number_of_copies)

In [ ]:
result

In [ ]:
result = pd.merge(result, dbbx, on="Fact_Check_Link", how = 'outer')

In [ ]:
sum(result.Number_of_copies)

In [ ]:
result = result.fillna(0)

In [ ]:
sum(result.Available_Number_of_copies)

In [ ]:
result['Copies_removed'] =  result ['Number_of_copies'] - result['Available_Number_of_copies']

In [ ]:
sum(result.Copies_removed)

In [ ]:
result

In [ ]:
db_auto_drop_duplicate.groupby('Fact_Check_Link')

In [ ]:
db_auto_drop_duplicate =db_auto_drop_duplicate.drop(columns = 'index')

In [ ]:
db_auto_drop_duplicate.Tweet_Time[0]

In [ ]:
result['First_Tweet_Text'] = ''
result['Last_Tweet_Text'] = ''
result['First_Tweet_ID'] =''
result['Last_Tweet_ID'] =''

In [ ]:
for i,r in result.iterrows():
    result.at[i,'First_Tweet_Time'] = db_auto_drop_duplicate.loc[db_auto_drop_duplicate.Fact_Check_Link 
                                                                 == result.Fact_Check_Link[i]].Tweet_Time.min()
    result.at[i,'Last_Tweet_Time'] = db_auto_drop_duplicate.loc[db_auto_drop_duplicate.Fact_Check_Link 
                                     == result.Fact_Check_Link[i]].Tweet_Time.max()   

In [ ]:
for i,r in result.iterrows():
    result.at[i, 'First_Tweet_Text'] = db_auto_drop_duplicate.loc[db_auto_drop_duplicate.Tweet_Time ==db_auto_drop_duplicate.loc[db_auto_drop_duplicate.Fact_Check_Link == result.Fact_Check_Link[i]].Tweet_Time.min()].Tweet_Text.to_string(index=False)
    result.at[i, 'First_Tweet_ID'] = db_auto_drop_duplicate.loc[db_auto_drop_duplicate.Tweet_Time ==db_auto_drop_duplicate.loc[db_auto_drop_duplicate.Fact_Check_Link == result.Fact_Check_Link[i]].Tweet_Time.min()].Tweet_ID.to_string(index=False)

    result.at[i, 'Last_Tweet_Text'] = db_auto_drop_duplicate.loc[db_auto_drop_duplicate.Tweet_Time ==db_auto_drop_duplicate.loc[db_auto_drop_duplicate.Fact_Check_Link == result.Fact_Check_Link[i]].Tweet_Time.max()].Tweet_Text.to_string(index=False)
    result.at[i, 'Last_Tweet_ID'] = db_auto_drop_duplicate.loc[db_auto_drop_duplicate.Tweet_Time ==db_auto_drop_duplicate.loc[db_auto_drop_duplicate.Fact_Check_Link == result.Fact_Check_Link[i]].Tweet_Time.max()].Tweet_ID.to_string(index=False)

In [ ]:
result.to_csv('Fake_News_Engagement.csv')

In [ ]:
result['Time_Difference'] = result['Last_Tweet_Time']-result['First_Tweet_Time']

In [ ]:
result['Time_Difference'] = result['Time_Difference'].dt.days + 1 

In [ ]:
db_auto_drop_duplicate=db_auto_drop_duplicate.reset_index(drop = True)

In [ ]:
db_auto_drop_duplicate.loc[db_auto_drop_duplicate.Tweet_Time ==db_auto_drop_duplicate.loc[db_auto_drop_duplicate.Fact_Check_Link 
                                                                 == result.Fact_Check_Link[10]].Tweet_Time.min()]

In [ ]:
merged_df = db_auto_drop_duplicate.merge(df_filter_all_remove_dup, how = 'inner', on = ['Tweet_ID'])

In [ ]:
merged_df.Tweet_ID

In [ ]:
result.groupby('Tweet_Topic').sum()

In [ ]:
result.to_csv('Fake_News_Engagement.csv')

In [ ]:
result.columns

In [ ]:
db

In [ ]:
fake_collected_news = db_auto_drop_duplicate.sort_values(by='Fact_Check_Link').reset_index(drop = True)

In [ ]:
fake_collected_news

In [ ]:
grp =fake_collected_news.groupby('Fact_Check_Link')

In [ ]:
fake_collected_news.drop(columns = ['ScreenName', 'SC']).to_csv('Collected_Fake_Tweets_Still_Available.csv')

In [ ]:
fake_collected_news.drop(columns = ['ScreenName', 'SC']).columns